<a href="https://colab.research.google.com/github/Segn11/zindi-financial-inclusion/blob/prediction4/financial_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import numpy as np

In [12]:
df = pd.read_csv('/content/Train (6).csv')
test = pd.read_csv('/content/Test (5).csv')

In [3]:
df.head()


,country,year,uniqueid,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,uniqueid_1,Yes,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed
1,Kenya,2018,uniqueid_2,No,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent
2,Kenya,2018,uniqueid_3,Yes,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed
3,Kenya,2018,uniqueid_4,No,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private
4,Kenya,2018,uniqueid_5,No,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23524 entries, 0 to 23523
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   country                 23524 non-null  object
 1   year                    23524 non-null  int64 
 2   uniqueid                23524 non-null  object
 3   bank_account            23524 non-null  object
 4   location_type           23524 non-null  object
 5   cellphone_access        23524 non-null  object
 6   household_size          23524 non-null  int64 
 7   age_of_respondent       23524 non-null  int64 
 8   gender_of_respondent    23524 non-null  object
 9   relationship_with_head  23524 non-null  object
 10  marital_status          23524 non-null  object
 11  education_level         23524 non-null  object
 12  job_type                23524 non-null  object
dtypes: int64(3), object(10)
memory usage: 2.3+ MB


In [13]:

df_id = df['uniqueid'].copy()
test_id = test["uniqueid"].copy()

# Prepare text data
X_df_raw = df.drop(columns=["uniqueid", "bank_account"]).copy()
X_te_raw = test.drop(columns=["uniqueid"]).copy()

# Combine for vectorization
all_data = pd.concat([X_df_raw, X_te_raw], axis=0).reset_index(drop=True)


In [ ]:
all_data.isna().sum()

,0
country,0
year,0
location_type,0
cellphone_access,0
household_size,0
age_of_respondent,0
gender_of_respondent,0
relationship_with_head,0
marital_status,0
education_level,0


In [ ]:
all_data.head()

,country,year,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed
1,Kenya,2018,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent
2,Kenya,2018,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed
3,Kenya,2018,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private
4,Kenya,2018,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed


In [14]:
cat_cols = all_data.select_dtypes(include="object").columns
cat_cols = list(cat_cols)
print(cat_cols)

['country', 'location_type', 'cellphone_access', 'gender_of_respondent', 'relationship_with_head', 'marital_status', 'education_level', 'job_type']


In [15]:
num_cols = all_data.select_dtypes(include="number").columns
num_cols = list(num_cols)
print(num_cols)

['year', 'household_size', 'age_of_respondent']


In [19]:
#feature engineering
edu_map = {
    "No formal education": 0,
    "Primary education": 1,
    "Secondary education": 2,
    "Tertiary education": 3
}
all_data["education_num"] = all_data["education_level"].map(edu_map)


In [20]:
#feature engineering
all_data["is_large_family"] = (all_data["household_size"] > 5).astype(int)
all_data["household_per_age"] = all_data["household_size"] / (all_data["age_of_respondent"] + 1)
all_data["rural_and_low_education"] = ((all_data["location_type"] == "Rural") & (all_data["education_level"].isin(["No formal education", "Primary education"]))).astype(int)



In [21]:
# Age weighted by household size
all_data["age_times_household"] = all_data["age_of_respondent"] * all_data["household_size"]

# Age and education interaction
all_data["age_times_education"] = all_data["age_of_respondent"] * all_data["education_num"]

# Large family in rural area
all_data["rural_large_family"] = all_data["is_large_family"] * (all_data["location_type"] == "Rural").astype(int)

# Age of respondent * rural & low education
all_data["age_rural_low_edu"] = all_data["age_of_respondent"] * all_data["rural_and_low_education"]
# Household size per education level (already exists as household_per_age)

all_data["household_per_education"] = all_data["household_size"] / (all_data["education_num"] + 1)

# Age per household member
all_data["age_per_person"] = all_data["age_of_respondent"] / (all_data["household_size"] + 1)
# Is adult (age > 25)
all_data["is_adult"] = (all_data["age_of_respondent"] > 25).astype(int)

# Is middle-aged (25-50)
all_data["is_middle_aged"] = ((all_data["age_of_respondent"] > 25) & (all_data["age_of_respondent"] <= 50)).astype(int)

# Rural AND no cellphone
all_data["rural_no_cellphone"] = ((all_data["location_type"] == "Rural") & (all_data["cellphone_access"] == "No")).astype(int)

# Married AND large family
all_data["married_large_family"] = ((all_data["marital_status"] == "Married/Living together") & (all_data["is_large_family"]==1)).astype(int)

#

In [22]:
all_data["economic_opportunity"] = (
    (all_data["cellphone_access"] == "Yes").astype(int) +
    (all_data["education_num"]) +
    (all_data["job_type"].isin(["Self employed", "Formally employed Private", "Formally employed Government"]).astype(int))
)

all_data["education_job_stability"] = all_data["education_num"] * all_data["job_type"].map({
    "Self employed": 2, "Formally employed Private": 3, "Formally employed Government": 3,
    "Casual labour": 1, "Informally employed": 1, "Other/Don't know/Refuse to answer": 0
})

all_data["rural_poverty"] = (
    (all_data["location_type"] == "Rural").astype(int) *
    (all_data["cellphone_access"] == "No").astype(int) *
    (all_data["education_num"] < all_data["education_num"].median()).astype(int)
)

all_data["dependency_ratio"] = all_data["household_size"] / (all_data["age_of_respondent"] + 1)

all_data["tech_exposure"] = (
    (all_data["cellphone_access"] == "Yes").astype(int) *
    (all_data["location_type"] == "Urban").astype(int)
)

all_data["married_and_educated"] = (
    (all_data["marital_status"] == "Married/Living together").astype(int) *
    (all_data["education_num"] > all_data["education_num"].median()).astype(int)
)

all_data["is_senior"] = (all_data["age_of_respondent"] > 50).astype(int)

all_data["tech_education"] = (
    (all_data["cellphone_access"] == "Yes").astype(int) *
    all_data["education_num"]
)

job_security_map = {
    "Formally employed Government": 3,
    "Formally employed Private": 2,
    "Self employed": 1,
    "Casual labour": 0,
    "Informally employed": 0,
    "Other/Don't know/Refuse to answer": -1
}
all_data["employment_security"] = all_data["job_type"].map(job_security_map)

all_data["socioeconomic_index"] = (
    all_data["education_num"] +
    all_data["employment_security"].fillna(0) +
    all_data["cellphone_access"].map({"Yes": 1, "No": 0}) +
    (all_data["location_type"] == "Urban").astype(int)
)



In [ ]:
# Show min, max, mean, quartiles
all_data[num_cols].describe()


,year,household_size,age_of_respondent
count,33610.000000,33610.000000,33610.000000
mean,2016.975930,3.791877,38.656114
std,0.847353,2.223138,16.447127
min,2016.000000,1.000000,16.000000
25%,2016.000000,2.000000,26.000000
50%,2017.000000,3.000000,35.000000
75%,2018.000000,5.000000,49.000000
max,2018.000000,21.000000,100.000000


In [ ]:
edu_map = {
    "No formal education": 0,
    "Primary education": 1,
    "Secondary education": 2,
    "Tertiary education": 3
}
all_data["education_num"] = all_data["education_level"].map(edu_map)


In [23]:
all_data.isna().sum()

,0
country,0
year,0
location_type,0
cellphone_access,0
household_size,0
age_of_respondent,0
gender_of_respondent,0
relationship_with_head,0
marital_status,0
education_level,0


In [24]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33610 entries, 0 to 33609
Data columns (total 35 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   country                  33610 non-null  object 
 1   year                     33610 non-null  int64  
 2   location_type            33610 non-null  object 
 3   cellphone_access         33610 non-null  object 
 4   household_size           33610 non-null  int64  
 5   age_of_respondent        33610 non-null  int64  
 6   gender_of_respondent     33610 non-null  object 
 7   relationship_with_head   33610 non-null  object 
 8   marital_status           33610 non-null  object 
 9   education_level          33610 non-null  object 
 10  job_type                 33610 non-null  object 
 11  education_num            32395 non-null  float64
 12  is_large_family          33610 non-null  int64  
 13  household_per_age        33610 non-null  float64
 14  rural_and_low_educatio

In [25]:
#handling missing values
all_data["education_num"].fillna(0, inplace=True)
all_data["age_times_education"].fillna(0, inplace=True)
all_data["household_per_education"].fillna(0, inplace=True)
all_data["socioeconomic_index"].fillna(0, inplace=True)
all_data["tech_education"].fillna(0, inplace=True)
all_data["employment_security"].fillna(0, inplace=True)
all_data["is_senior"].fillna(0, inplace=True)
all_data["education_job_stability"].fillna(0, inplace=True)
all_data["economic_opportunity"].fillna(0, inplace=True)
#all_data["rural_poverty"].fillna(0, inplace=True)

/tmp/ipython-input-2831452206.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_data["education_num"].fillna(0, inplace=True)
/tmp/ipython-input-2831452206.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try u

In [26]:
#feature selection
all_data.drop(columns=["year"], inplace=True)


In [27]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33610 entries, 0 to 33609
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   country                  33610 non-null  object 
 1   location_type            33610 non-null  object 
 2   cellphone_access         33610 non-null  object 
 3   household_size           33610 non-null  int64  
 4   age_of_respondent        33610 non-null  int64  
 5   gender_of_respondent     33610 non-null  object 
 6   relationship_with_head   33610 non-null  object 
 7   marital_status           33610 non-null  object 
 8   education_level          33610 non-null  object 
 9   job_type                 33610 non-null  object 
 10  education_num            33610 non-null  float64
 11  is_large_family          33610 non-null  int64  
 12  household_per_age        33610 non-null  float64
 13  rural_and_low_education  33610 non-null  int64  
 14  age_times_household   

In [ ]:
num_cols = all_data.select_dtypes(include=[np.number]).columns

# Compute correlation matrix
corr_matrix = all_data[num_cols].corr().abs()  # absolute value

# Find highly correlated pairs
high_corr = np.where(corr_matrix > 0.9)
high_corr_pairs = [(num_cols[i], num_cols[j], corr_matrix.iloc[i, j])
                   for i, j in zip(*high_corr) if i < j]

print("Highly correlated feature pairs (corr>0.9):")
for f1, f2, corr_val in high_corr_pairs:
    print(f"{f1} - {f2}: {corr_val:.2f}")

Highly correlated feature pairs (corr>0.9):


In [28]:
num_cols = all_data.select_dtypes(include="number").columns
skewed = all_data[num_cols].apply(lambda x: x.dropna().skew()).abs()
skew_cols = skewed[skewed > 0.75].index
all_data[skew_cols] = np.log1p(all_data[skew_cols])

In [ ]:
all_data.head()

,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,education_num,is_large_family,household_per_age,rural_and_low_education,age_times_household,...,job_type_Government Dependent,job_type_Informally employed,job_type_No Income,job_type_Other Income,job_type_Remittance Dependent,job_type_Self employed,country_Kenya,country_Rwanda,country_Tanzania,country_Uganda
0,0,1,1.386294,3.218876,0,2.0,0.000000,0.113329,0,4.290459,...,False,False,False,False,False,True,True,False,False,False
1,0,0,1.791759,4.262680,0,0.0,0.000000,0.068053,1,5.860786,...,True,False,False,False,False,False,True,False,False,False
2,1,1,1.791759,3.295837,1,0.0,0.000000,0.169899,0,4.875197,...,False,False,False,False,False,True,True,False,False,False
3,0,1,1.791759,3.555348,0,1.0,0.000000,0.133531,1,5.141664,...,False,False,False,False,False,False,True,False,False,False
4,1,0,2.197225,3.295837,1,1.0,0.693147,0.259511,0,5.342334,...,False,True,False,False,False,False,True,False,False,False


In [ ]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33610 entries, 0 to 33609
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   country                  33610 non-null  object 
 1   location_type            33610 non-null  object 
 2   cellphone_access         33610 non-null  object 
 3   household_size           33610 non-null  float64
 4   age_of_respondent        33610 non-null  float64
 5   gender_of_respondent     33610 non-null  object 
 6   relationship_with_head   33610 non-null  object 
 7   marital_status           33610 non-null  object 
 8   education_level          33610 non-null  object 
 9   job_type                 33610 non-null  object 
 10  education_num            33610 non-null  float64
 11  is_large_family          33610 non-null  int64  
 12  household_per_age        33610 non-null  float64
 13  rural_and_low_education  33610 non-null  int64  
 14  age_times_household   

In [29]:
from sklearn.preprocessing import LabelEncoder
#float_array = all_data[["household_size", "age_of_respondent", "year"]].values.astype(float)

# List of categorical columns for One-Hot Encoding
cat_cols = [
    "relationship_with_head",
    "marital_status",
    "education_level",
    "job_type",
    "country",

]

# Keep only columns that exist in all_data
cat_cols = [col for col in cat_cols if col in all_data.columns]

# One-Hot Encoding
all_data = pd.get_dummies(all_data, columns=cat_cols, prefix_sep="_")

# Label encoding for small/binary categorical columns
le = LabelEncoder()
for col in ["location_type", "cellphone_access", "gender_of_respondent"]:
    if col in all_data.columns:
        all_data[col] = le.fit_transform(all_data[col].astype(str))


In [30]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33610 entries, 0 to 33609
Data columns (total 60 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   location_type                                    33610 non-null  int64  
 1   cellphone_access                                 33610 non-null  int64  
 2   household_size                                   33610 non-null  float64
 3   age_of_respondent                                33610 non-null  float64
 4   gender_of_respondent                             33610 non-null  int64  
 5   education_num                                    33610 non-null  float64
 6   is_large_family                                  33610 non-null  float64
 7   household_per_age                                33610 non-null  float64
 8   rural_and_low_education                          33610 non-null  int64  
 9   age_times_household         

In [31]:
# If train['bank_account'] contains 'Yes'/'No'
y = df['bank_account'].map({'No': 0, 'Yes': 1})


In [32]:
n_train = X_df_raw.shape[0]
X_train = all_data.iloc[:n_train, :].copy()
X_test  = all_data.iloc[n_train:, :].copy()

In [33]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y, test_size=0.2, random_state=42
)



In [34]:
from lightgbm import LGBMClassifier
from sklearn.metrics import mean_absolute_error, roc_auc_score

model = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=7,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(X_tr, y_tr)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2670, number of negative: 16149
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004772 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1493
[LightGBM] [Info] Number of data points in the train set: 18819, number of used features: 59
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.141878 -> initscore=-1.799780
[LightGBM] [Info] Start training from score -1.799780
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

LGBMClassifier(colsample_bytree=0.8, learning_rate=0.05, max_depth=7,
               n_estimators=500, random_state=42, subsample=0.8)

In [35]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 6.2 MB/s eta 0:00:00


In [36]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 6.0 MB/s eta 0:00:00


In [37]:
# ---------------------------------------------
# 0️⃣ Imports
# ---------------------------------------------
import optuna
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score, mean_absolute_error
from sklearn.model_selection import train_test_split

# ---------------------------------------------
# 1️⃣ Split Data (if not already done)
# ---------------------------------------------
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y, test_size=0.2, random_state=42, stratify=y
)

# ---------------------------------------------
# 2️⃣ Define Optuna Objective Function
# ---------------------------------------------
def objective(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 300, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "depth": trial.suggest_int("depth", 4, 10),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-3, 10.0, log=True),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        "random_strength": trial.suggest_float("random_strength", 0.0, 2.0),
        "border_count": trial.suggest_int("border_count", 32, 255),
        "random_seed": 42,
        "eval_metric": "AUC",
        "verbose": 0,
        "auto_class_weights": "Balanced",  # handles imbalance in target
    }

    model = CatBoostClassifier(**params)
    model.fit(
        X_tr,
        y_tr,
        eval_set=(X_val, y_val),
        early_stopping_rounds=50,
        use_best_model=True
    )

    preds = model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, preds)
    return auc

# ---------------------------------------------
# 3️⃣ Run Optuna Study
# ---------------------------------------------
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)  # Increase trials for deeper tuning

# ---------------------------------------------
# 4️⃣ Train Final Model with Best Parameters
# ---------------------------------------------
best_params = study.best_params
print("✅ Best Parameters:", best_params)

model = CatBoostClassifier(**best_params)
model.fit(
    X_tr,
    y_tr,
    eval_set=(X_val, y_val),
    early_stopping_rounds=50,
    use_best_model=True,
)

# ---------------------------------------------
# 5️⃣ Evaluate on Validation Data
# ---------------------------------------------
preds = model.predict_proba(X_val)[:, 1]
val_mae = mean_absolute_error(y_val, preds)
auc = roc_auc_score(y_val, preds)

print(f"Validation MAE: {val_mae:.4f}")
print(f"Validation ROC AUC: {auc:.4f}")


[I 2025-10-21 14:56:41,184] A new study created in memory with name: no-name-403ade07-1812-444a-8e83-0081ef84f547
[I 2025-10-21 14:56:47,097] Trial 0 finished with value: 0.8767163490961587 and parameters: {'iterations': 341, 'learning_rate': 0.040812735076681, 'depth': 9, 'l2_leaf_reg': 4.453845001186276, 'bagging_temperature': 0.17993598599323535, 'random_strength': 0.735845144861486, 'border_count': 136}. Best is trial 0 with value: 0.8767163490961587.
[I 2025-10-21 14:57:02,054] Trial 1 finished with value: 0.8677414545897464 and parameters: {'iterations': 520, 'learning_rate': 0.02049779438670338, 'depth': 10, 'l2_leaf_reg': 0.038618700420978674, 'bagging_temperature': 0.21141486673527032, 'random_strength': 0.11934145274469588, 'border_count': 41}. Best is trial 0 with value: 0.8767163490961587.
[I 2025-10-21 14:57:08,078] Trial 2 finished with value: 0.8753716654723058 and parameters: {'iterations': 473, 'learning_rate': 0.024965906550650296, 'depth': 6, 'l2_leaf_reg': 0.1141644

✅ Best Parameters: {'iterations': 429, 'learning_rate': 0.022318684643420718, 'depth': 7, 'l2_leaf_reg': 4.793005272470566, 'bagging_temperature': 0.8932596100773837, 'random_strength': 1.1574553584679084, 'border_count': 144}
0:	learn: 0.6703254	test: 0.6703894	best: 0.6703894 (0)	total: 9.52ms	remaining: 4.07s
1:	learn: 0.6508530	test: 0.6507119	best: 0.6507119 (1)	total: 18.6ms	remaining: 3.97s
2:	learn: 0.6325669	test: 0.6323391	best: 0.6323391 (2)	total: 27.3ms	remaining: 3.88s
3:	learn: 0.6152746	test: 0.6150542	best: 0.6150542 (3)	total: 36ms	remaining: 3.82s
4:	learn: 0.5986836	test: 0.5984639	best: 0.5984639 (4)	total: 44.8ms	remaining: 3.8s
5:	learn: 0.5810070	test: 0.5809046	best: 0.5809046 (5)	total: 53.4ms	remaining: 3.77s
6:	learn: 0.5668875	test: 0.5667513	best: 0.5667513 (6)	total: 62.3ms	remaining: 3.75s
7:	learn: 0.5536721	test: 0.5534732	best: 0.5534732 (7)	total: 68.8ms	remaining: 3.62s
8:	learn: 0.5403575	test: 0.5401758	best: 0.5401758 (8)	total: 78ms	remaining: 3

In [ ]:
y_val_pred = model.predict(X_val)
y_val_prob = model.predict_proba(X_val)[:, 1]  # probability of class 1

# Since Zindi uses MAE
val_mae = mean_absolute_error(y_val, y_val_pred)
print("Validation MAE:", val_mae)

# Optional: ROC-AUC for additional insight
roc_auc = roc_auc_score(y_val, y_val_prob)
print("Validation ROC-AUC:", roc_auc)


Validation MAE: 0.09436769394261424
Validation ROC-AUC: 0.9136123903684933


In [38]:
model.fit(X_train, y)  # train on all training data
test_preds = model.predict(X_test)




0:	learn: 0.6699986	total: 10.8ms	remaining: 4.62s
1:	learn: 0.6504319	total: 20.9ms	remaining: 4.46s
2:	learn: 0.6315949	total: 31ms	remaining: 4.4s
3:	learn: 0.6134474	total: 41.1ms	remaining: 4.36s
4:	learn: 0.5967172	total: 51.2ms	remaining: 4.34s
5:	learn: 0.5801100	total: 61.3ms	remaining: 4.32s
6:	learn: 0.5655114	total: 70.8ms	remaining: 4.27s
7:	learn: 0.5519193	total: 80.7ms	remaining: 4.25s
8:	learn: 0.5391129	total: 90.5ms	remaining: 4.22s
9:	learn: 0.5253763	total: 100ms	remaining: 4.19s
10:	learn: 0.5136631	total: 112ms	remaining: 4.26s
11:	learn: 0.5015462	total: 122ms	remaining: 4.25s
12:	learn: 0.4909357	total: 132ms	remaining: 4.22s
13:	learn: 0.4794070	total: 142ms	remaining: 4.21s
14:	learn: 0.4705037	total: 153ms	remaining: 4.21s
15:	learn: 0.4617222	total: 163ms	remaining: 4.2s
16:	learn: 0.4546199	total: 168ms	remaining: 4.08s
17:	learn: 0.4463494	total: 178ms	remaining: 4.07s
18:	learn: 0.4376302	total: 189ms	remaining: 4.07s
19:	learn: 0.4292404	total: 198ms	re

In [ ]:
# Create submission DataFrame
submission = pd.DataFrame({"uniqueid": test["uniqueid"] + " x " + test["country"],
                           "bank_account": test.bank_account})

In [39]:
# test_id: Series of unique IDs
# test_country: Series of country names corresponding to test IDs
test_country = test["country"].copy()  # make sure you have this column

# Prepare submission
submission = pd.DataFrame({
    "unique_id": test_id.astype(str) + " x " + test_country.astype(str),
    "bank_account": test_preds
})

submission.to_csv("submission.csv", index=False)
print("✅ Submission saved!")


✅ Submission saved!


In [40]:
submission.head()

,unique_id,bank_account
0,uniqueid_6056 x Kenya,1
1,uniqueid_6060 x Kenya,1
2,uniqueid_6065 x Kenya,0
3,uniqueid_6072 x Kenya,0
4,uniqueid_6073 x Kenya,0


In [41]:
from google.colab import files
files.download("submission.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>